In [1]:
import os
import glob
import numpy as np
import mne

# Path to raw .vhdr EEG files
eeg_data = '/storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata'

# Output directory for processed .npz files
output_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data2'
os.makedirs(output_dir, exist_ok=True)

# Get all .vhdr files
all_vhdr_files = glob.glob(os.path.join(eeg_data, '*.vhdr'))

# Filter specific files
vhdr_paths = [
    f for f in all_vhdr_files
    if 'Move' not in os.path.basename(f)
    and 'session3_sub13' not in os.path.basename(f)
    and 'session1_sub23' not in os.path.basename(f)
    and 'session2_sub7' not in os.path.basename(f)
    and 'session2_sub12' not in os.path.basename(f)
    and 'session2_sub6' not in os.path.basename(f)
]

# Motor imagery event labels
wrist_events = {'pronation': 91, 'supination': 101}

# Process and save each file one-by-one
for i, vhdr_path in enumerate(vhdr_paths):
    try:
        print(f"Processing: {vhdr_path}")

        raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)

        # Drop EOG & EMG channels
        raw.drop_channels([ch for ch in raw.ch_names if 'EOG' in ch or 'EMG' in ch])

        # Band-pass filter for MI (8–30 Hz)
        raw.filter(1., 30., fir_design='firwin')

        # Extract epochs
        events, event_id = mne.events_from_annotations(raw)
        # epochs = mne.Epochs(
        #     raw, events, event_id=wrist_events,
        #     tmin=-0.2, tmax=1.0,
        #     event_repeated="merge",
        #     baseline=(None, 0),
        #     preload=True
        # )

        epochs = mne.Epochs(
            raw, events, event_id=wrist_events,
            tmin=-0.2, tmax=4.0,    # Capture full MI window with baseline
            baseline=(None, 0),     # Baseline correction using pre-0s
            preload=True
        )

        X = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
        y = epochs.events[:, -1]
        y = np.array([0 if label == 91 else 1 for label in y])  # Map labels

        # Save as .npz
        vhdr_name = os.path.splitext(os.path.basename(vhdr_path))[0]
        output_path = os.path.join(output_dir, f"{vhdr_name}.npz")
        np.savez_compressed(output_path, X=X, y=y)
        print(f"Saved: {output_path} (X={X.shape}, y={y.shape})\n")

        # Optional: clear variables to release memory
        del raw, events, event_id, epochs, X, y

    except Exception as e:
        print(f"Failed processing {vhdr_path}: {e}")


Processing: /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr
Extracting parameters from /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr...
Setting channel info structure...


/tmp/ipykernel_316151/825814001.py:35: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_316151/825814001.py:35: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_316151/825814001.py:35: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (1250.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Reading 0 ... 2564199  =      0.000 ...  1025.680 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 8251 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Used Annotations descriptions: [np.str_('Comment/actiCAP Active Shield On'), np.str_('New Segment/'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S  9'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 13'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 91'), np.str_('Stimulus/S101')]
Not setting metadata
100 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 10501 original time points ...


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 8251 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Used Annotations descriptions: [np.str_('Comment/actiCAP Active Shield On'), np.str_('New Segment/'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S  9'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 13'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 91'), np.str_('Stimulus/S101')]
Not setting metadata
100 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 10501 original time points ...


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    2.5s finished


0 bad epochs dropped
Saved: /storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data2/session2_sub17_twist_MI.npz (X=(100, 60, 10501), y=(100,))

